# Data Loaders

In [12]:
del train_data

In [13]:
from signjoey.helpers import (
    load_config,
    load_checkpoint,
)
from signjoey.data import load_data, make_data_iter
from signjoey.model import build_model
from signjoey.batch import Batch
from torchtext.data import Dataset
from signjoey.vocabulary import (
    TextVocabulary,
    GlossVocabulary,
    PAD_TOKEN,
    EOS_TOKEN,
    BOS_TOKEN,
)

In [14]:
cfg_file = 'configs/example.yaml'
ckpt = 'output/best-kaggle.ckpt'

cfg = load_config(cfg_file)

In [15]:
train_data, dev_data, test_data, gls_vocab, txt_vocab = load_data(data_cfg=cfg["data"])
del train_data, test_data

In [16]:
dev_data.fields

{'sequence': <torchtext.data.field.RawField at 0x172fa0c9390>,
 'signer': <torchtext.data.field.RawField at 0x172fa0c8550>,
 'sgn': <torchtext.data.field.Field at 0x172fa0c8880>,
 'gls': <torchtext.data.field.Field at 0x172fa0c8e20>,
 'txt': <torchtext.data.field.Field at 0x172fa0c8400>}

In [17]:
dev_data[0].__dict__.values()

dict_values(['dev/11August_2010_Wednesday_tagesschau-2', 'Signer08', [tensor([4.1970e-01, 1.7552e+00, 2.4212e-01,  ..., 1.0000e-08, 1.0000e-08,
        1.0000e-08]), tensor([2.3592e+00, 2.2639e+00, 4.4332e-01,  ..., 1.0000e-08, 1.0000e-08,
        1.0000e-08]), tensor([5.0251e+00, 2.1903e+00, 1.3583e-01,  ..., 5.7079e-02, 1.0000e-08,
        1.0000e-08]), tensor([5.3371e+00, 1.3213e+00, 1.0000e-08,  ..., 2.1011e+00, 1.0000e-08,
        1.0000e-08]), tensor([4.2258e+00, 2.5479e+00, 1.0000e-08,  ..., 8.2129e-01, 1.0000e-08,
        1.0000e-08]), tensor([3.6651e+00, 3.7671e+00, 1.0000e-08,  ..., 3.9290e-01, 1.0000e-08,
        1.0818e-01]), tensor([3.1189e+00, 3.1486e+00, 1.0000e-08,  ..., 1.8473e-01, 1.0000e-08,
        1.0979e+00]), tensor([4.0009e+00, 3.6177e+00, 1.0000e-08,  ..., 1.0000e-08, 1.0000e-08,
        2.3070e+00]), tensor([4.9496e+00, 3.4256e+00, 1.0000e-08,  ..., 1.0000e-08, 1.0000e-08,
        2.5931e+00]), tensor([6.0652e+00, 3.4525e+00, 4.0531e-02,  ..., 1.0000e-08, 1.00

In [18]:

train_config = cfg["training"]

valid_iter = make_data_iter(
    dev_data,
    batch_size=train_config['batch_size'],
    batch_type=train_config['batch_type'],
    train=False,
    shuffle=False,
)

In [19]:
frame_subsampling_ratio = cfg["data"].get(
    "frame_subsampling_ratio", None
)
random_frame_subsampling = cfg["data"].get(
    "random_frame_subsampling", None
)
random_frame_masking_ratio = cfg["data"].get(
    "random_frame_masking_ratio", None
)
txt_pad_idx = txt_vocab.stoi[PAD_TOKEN]

In [20]:
for batch in iter(valid_iter):
    # reactivate training
    # create a Batch object from torchtext batch
    batch = Batch(
        is_train=False,
        torch_batch=batch,
        txt_pad_index= txt_pad_idx,
        sgn_dim=cfg['data']['feature_size'],
        use_cuda=False,
        frame_subsampling_ratio=frame_subsampling_ratio,
        random_frame_subsampling=random_frame_subsampling,
        random_frame_masking_ratio=random_frame_masking_ratio,
    )
    break

d:\CS4203 - FYP\project folder\Signify\strf\lib\site-packages\torchtext\data\field.py:359: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)


In [37]:
batch.sgn[0].shape

torch.Size([165, 1024])

In [34]:
len(batch.sgn_mask), len(batch.sgn_mask[0][0]) 

(32, 165)

In [35]:
batch.sgn_mask[0][0]

tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, 

In [52]:
import pickle
import gzip
filename = 'data/phoenix14t.pami0.dev'


with gzip.open(filename, "rb") as f:
    loaded_object = pickle.load(f)

In [57]:
loaded_object[0]['sign'].shape

torch.Size([42, 1024])

In [77]:
import torch
i = -1

(loaded_object[0]['sign'][i] != torch.zeros(loaded_object[0]['sign'][i].shape)).sum()

tensor(640)

# Load Model

In [39]:
model_checkpoint = load_checkpoint(ckpt, use_cuda=False)

do_recognition = cfg["training"].get("recognition_loss_weight", 1.0) > 0.0
do_translation = cfg["training"].get("translation_loss_weight", 1.0) > 0.0

model = build_model(
    cfg=cfg["model"],
    gls_vocab=gls_vocab,
    txt_vocab=txt_vocab,
    sgn_dim=sum(cfg["data"]["feature_size"])
    if isinstance(cfg["data"]["feature_size"], list)
    else cfg["data"]["feature_size"],
    do_recognition=do_recognition,
    do_translation=do_translation,
    ensemble=False,
    ensembleN=1
)

512
512
512
512
512
512


In [40]:
model.load_state_dict(model_checkpoint["model_state"])

<All keys matched successfully>

In [44]:
model.sgn_embed

SpatialEmbeddings(embedding_dim=512, input_size=1024)

In [42]:
embeddings = model.sgn_embed(batch.sgn, batch.sgn_mask)

In [43]:
embeddings.shape

torch.Size([32, 165, 512])

In [46]:
output = model.encoder(embeddings, batch.sgn_mask)

In [48]:
output[0]

tensor([[[ 0.2181,  0.5392,  0.2666,  ...,  1.9912, -0.2534, -0.1238],
         [ 0.3249, -0.1509, -0.1520,  ...,  1.1652, -0.1434, -0.0041],
         [ 0.3100, -0.1471, -0.3730,  ...,  0.5793, -1.0380,  1.2287],
         ...,
         [-0.1267, -0.1866,  1.0359,  ...,  0.8788, -0.9316, -0.5963],
         [-0.5408,  0.4729,  0.1245,  ...,  0.9377, -0.9623, -0.9559],
         [ 0.4522, -0.1276,  0.4823,  ...,  0.0464, -1.4490, -1.3038]],

        [[-0.0720, -0.3846, -0.0576,  ...,  0.0783, -0.3989, -0.2873],
         [-0.0642, -0.6298,  0.1268,  ...,  0.0550, -0.0311, -0.1896],
         [-0.0707, -1.0677,  0.3072,  ...,  0.1430, -0.4266, -0.3633],
         ...,
         [-0.0641,  0.4085, -0.3759,  ...,  0.6881, -0.3928, -0.4809],
         [-0.2206, -0.2361, -0.1940,  ...,  0.0339, -1.2790, -0.4826],
         [ 0.5658, -0.4308, -0.1457,  ...,  0.3657, -1.2463,  0.0250]],

        [[-0.0565, -0.3880, -0.7907,  ...,  0.4861,  0.4019,  0.4380],
         [ 0.4766, -0.4889, -0.2924,  ...,  0

In [50]:
output[0].shape

torch.Size([32, 165, 512])